In [56]:
iscan_file = '/NHP_MRI/NHP-BIDS/projects/NaturalMovie/sub-eddy/ses-20191114/func/sub-eddy_ses-20191114_task-naturalmovie_run-02_recording-eyetrace_physio.tda'

In [57]:
iscan_parts = iscan_file.strip().split("/")
eyefile = iscan_parts[-1].rsplit('.',1)[0]
print(eyefile)

sub-eddy_ses-20191114_task-naturalmovie_run-02_recording-eyetrace_physio


In [75]:
import pandas as pd

GetRuns = False
run = -1
runs={}

with open(iscan_file, "r", encoding="utf-8") as file:
    all_lines = file.readlines()

nlines = len(all_lines)

line_cnt = 0;
with open(iscan_file, "r", encoding="utf-8") as file:   
    for line in file:
        line_cnt = line_cnt+1
        data = line.strip().split("\t")  # Split each line by tab
        #print(columns)
                
        if data[0] == 'DATA INFO':
            GetRuns = True
            AddData = False
            print('Start getting data')
        
        if GetRuns:
            if len(data)>1:
                if data[1] == 'Pupil H1 ':
                    Hdr = data
                    run = run+1 
                    datacollect = [Hdr]
                    AddData = True
                    print('Header detected')

                elif AddData & (data[0] != 'Sample #'):
                    datacollect.append(data)
                    # print('Adding data')
                    
            elif AddData:
                AddData = False
                print('End of run detected')
                runs[run] = pd.DataFrame(datacollect[1:], columns=datacollect[0])

            if line_cnt == nlines:
                AddData = False
                print('End of run detected')
                runs[run] = pd.DataFrame(datacollect[1:], columns=datacollect[0])


Start getting data
Header detected
End of run detected
Header detected
End of run detected
Header detected
End of run detected


In [76]:
print(len(runs))
runs[2].head(5)

3


,Run 3:,Pupil H1,Pupil V1,Pupil D1,P-CR H1,P-CR V1,Dig Inp 1A
0,146457,191.90,128.00,197.00,-12.00,-13.10,14.00
1,146458,192.10,128.00,197.00,-12.10,-13.10,14.00
2,146459,192.50,128.60,196.00,-12.00,-13.00,14.00
3,146460,194.90,131.60,196.00,-10.80,-11.30,14.00
4,146461,199.30,133.80,197.00,-9.10,-10.50,14.00


In [78]:
# save as csv files
for r in runs:
    runs[r].to_csv(eyefile + '_eye-run-' + str(r+1) + '.csv', index=False)